In [ ]:
import sys; sys.path.append("../src")

from crnverifier.utils import parse_crn
from compile import *
from crnverifier import crn_bisimulation_test
from pprint import pprint
from dsdobjects import ReactionS
import dsdobjects.objectio as dsdio

# TODO: instead convert dirctly to list format
def crn_format(reaction: ReactionS) -> str:
    return " + ".join(str(s) for s in reaction.reactants) + " -> " + " + ".join(str(s) for s in reaction.products)

In [3]:
# A formal CRN and a corresponding implementation CRN
fcrn = "A -> B"
icrn = "a1 -> b1; x -> a1; x -> b1; y -> b1; y -> a1; x -> a0; a0 -> a1"

# A quick interface to the internal list representation of CRNs, 
# the first CRN contains only formal species (fs), the species
# of the second CRN are not important right now.
fcrn, fs = parse_crn(fcrn, is_file = False)
icrn, ims = parse_crn(icrn, is_file = False)

In [11]:
# For the other notions, we may need a (partial) interpretation:
inter = {'a0': 'A',
         'a1': 'A',
         'b1': 'B'}

# Test if there exists a correct CRN bisimulation that interprets 
# the reactions of icrn as reactions of fcrn.
# The (partial) interpretation is optional here.
v, i = crn_bisimulation_test(fcrn, icrn, fs)#, interpretation = inter)
print(v, i)

True {'a1': ['A'], 'b1': ['B'], 'y': ['A'], 'a0': ['A'], 'x': ['A']}


In [12]:
from crnverifier import integrated_hybrid_test, compositional_hybrid_test


In [13]:
# Verify whether icrn is a correct implementation of fcrn using the 
# two supported hybrid notions.
# The (partial) interpretation is required here.
v, i = integrated_hybrid_test(fcrn, icrn, fs, inter)
print(v, i)
v, i = compositional_hybrid_test(fcrn, icrn, fs, inter)
print(v, i)

True {'a0': 'A', 'a1': 'A', 'b1': 'B'}
True {'a0': 'A', 'a1': 'A', 'b1': 'B'}


## signal reversal

In [1]:
N = 3


In [ ]:

fcrn, fs = parse_crn("; ".join(f"X{i} -> Y{j}" for i in range(N) for j in range(N) if i != j), is_file = False)
fcrn

In [5]:
# alternative: add gates to fCRN (OPT: also update interp)
fcrn, fs = parse_crn("; ".join(f"X{i} + ReversalGate_X{i}_Y{j} -> Y{j}" for i in range(N) for j in range(N) if i != j), is_file = False)
fcrn

[[['X0', 'ReversalGate_X0_Y1'], ['Y1']],
 [['X0', 'ReversalGate_X0_Y2'], ['Y2']],
 [['X1', 'ReversalGate_X1_Y0'], ['Y0']],
 [['X1', 'ReversalGate_X1_Y2'], ['Y2']],
 [['X2', 'ReversalGate_X2_Y0'], ['Y0']],
 [['X2', 'ReversalGate_X2_Y1'], ['Y1']]]

In [2]:
sigrev = DSDCircuit(new_toehold_generation=True)
input_signal = sigrev.new_signal('X', N)
output_signal = sigrev.new_signal('Y', N)
sigrev.add_global(TH_EXT_NAME, 2)

sigrev.add_modules([
    SignalReversal(input_signal, output_signal, reversible_design=False)
], add_reporting=False)

Output species not implemented for the last module; outputs not set.


In [4]:
fcrn = [r.list_format() for r in sigrev.formal_CRN(include_supporting_species=True)]
fcrn

[[['X0', 'ReversalGate_0_1'], ['Y1']],
 [['X0', 'ReversalGate_0_2'], ['Y2']],
 [['X1', 'ReversalGate_1_0'], ['Y0']],
 [['X1', 'ReversalGate_1_2'], ['Y2']],
 [['X2', 'ReversalGate_2_0'], ['Y0']],
 [['X2', 'ReversalGate_2_1'], ['Y1']]]

In [8]:
dsd_path = f"../artifacts/signal-reversal/{N}-signal-reversal_WVtest.pil"
sigrev.export_PIL(output_file=dsd_path, name_intermediates=True)

No outputs defined for the circuit; output species metadata will be empty.


- [ ] run enumeration!

In [ ]:

dsdio.set_io_objects()
pildata = dsdio.read_pil(os.path.splitext(dsd_path)[0] + "_ENUM-CRNcond_vmax10.pil", is_file=True)
#pildata.keys()

In [13]:
det_icrn, det_is = parse_crn(";".join(map(crn_format, pildata["det_reactions"])), is_file = False)
con_icrn, con_is = parse_crn(";".join(map(crn_format, pildata["con_reactions"])), is_file = False)

In [23]:
partial_interp = {} # mapping from signal species to formal signal names (singleton lists)

# inputs
fst_module = sigrev._modules[0]
assert isinstance(fst_module, SignalReversal)
for i in range(fst_module.input.dim):
    partial_interp[fst_module.input_species_label(i)] = [f"{fst_module.input.recognition_domain_prefix.upper()}{i}"]

# outputs of all modules (intermediates and final outputs)
for mod in sigrev._modules:
    if mod.output is None: continue

    for i in range(mod.output.dim):
        for out_cpx in mod.all_output_signal_species(i):
            partial_interp[out_cpx.name] = [f"{mod.output.recognition_domain_prefix.upper()}{i}"]

partial_interp

{'InputSignal_X0': ['X0'],
 'InputSignal_X1': ['X1'],
 'InputSignal_X2': ['X2'],
 'Y0_from_X1': ['Y0'],
 'Y0_from_X2': ['Y0'],
 'Y1_from_X0': ['Y1'],
 'Y1_from_X2': ['Y1'],
 'Y2_from_X0': ['Y2'],
 'Y2_from_X1': ['Y2']}

In [4]:
sigrev.partial_species_interpretation()

{'InputSignal_X0': ['X0'],
 'InputSignal_X1': ['X1'],
 'InputSignal_X2': ['X2'],
 'Y0_from_X1': ['Y0'],
 'Y0_from_X2': ['Y0'],
 'Y1_from_X0': ['Y1'],
 'Y1_from_X2': ['Y1'],
 'Y2_from_X0': ['Y2'],
 'Y2_from_X1': ['Y2']}

In [24]:
all([set(v) & fs == set(v) for v in partial_interp.values()])

True

In [25]:
fcrn

[[['X0', 'ReversalGate_X0_Y1'], ['Y1']],
 [['X0', 'ReversalGate_X0_Y2'], ['Y2']],
 [['X1', 'ReversalGate_X1_Y0'], ['Y0']],
 [['X1', 'ReversalGate_X1_Y2'], ['Y2']],
 [['X2', 'ReversalGate_X2_Y0'], ['Y0']],
 [['X2', 'ReversalGate_X2_Y1'], ['Y1']]]

In [26]:
con_icrn

[[['ReversalGate_X2_Y0', 'InputSignal_X2'], ['e26', 'Y0_from_X2']],
 [['ReversalGate_X1_Y2', 'InputSignal_X1'], ['e12', 'Y2_from_X1']],
 [['ReversalGate_X2_Y1', 'InputSignal_X2'], ['e26', 'Y1_from_X2']],
 [['ReversalGate_X0_Y1', 'InputSignal_X0'], ['e47', 'Y1_from_X0']],
 [['ReversalGate_X0_Y2', 'InputSignal_X0'], ['e47', 'Y2_from_X0']],
 [['ReversalGate_X1_Y0', 'InputSignal_X1'], ['e12', 'Y0_from_X1']]]

In [30]:
from pprint import pprint

icrn = det_icrn
print("check bisimulation", len(fcrn), len(icrn))
v, full_interp = crn_bisimulation_test(icrn=icrn, fcrn=fcrn, interpretation = partial_interp, formals = None) # formals inferred
if not v:
    print("NOT equivalent")
else:
    print("EQUIVALENT")
    pprint(full_interp)

check bisimulation 6 42
{'InputSignal_X0': ['X0'],
 'InputSignal_X1': ['X1'],
 'InputSignal_X2': ['X2'],
 'ReversalGate_X0_Y1': ['ReversalGate_X0_Y1'],
 'ReversalGate_X0_Y2': ['ReversalGate_X0_Y2'],
 'ReversalGate_X1_Y0': ['ReversalGate_X1_Y0'],
 'ReversalGate_X1_Y2': ['ReversalGate_X1_Y2'],
 'ReversalGate_X2_Y0': ['ReversalGate_X2_Y0'],
 'ReversalGate_X2_Y1': ['ReversalGate_X2_Y1'],
 'Y0_from_X1': ['Y0'],
 'Y0_from_X2': ['Y0'],
 'Y1_from_X0': ['Y1'],
 'Y1_from_X2': ['Y1'],
 'Y2_from_X0': ['Y2'],
 'Y2_from_X1': ['Y2'],
 'e10': ['ReversalGate_X1_Y2', 'X1'],
 'e12': [],
 'e14': ['ReversalGate_X2_Y1', 'X2'],
 'e16': ['ReversalGate_X0_Y1', 'X2'],
 'e18': ['ReversalGate_X2_Y0', 'X2'],
 'e2': ['ReversalGate_X2_Y1', 'X1'],
 'e20': ['ReversalGate_X0_Y2', 'X2'],
 'e22': ['ReversalGate_X1_Y2', 'X2'],
 'e26': [],
 'e30': ['ReversalGate_X1_Y0', 'X1'],
 'e32': ['ReversalGate_X1_Y0', 'X2'],
 'e35': ['ReversalGate_X2_Y1', 'X0'],
 'e37': ['ReversalGate_X0_Y1', 'X0'],
 'e39': ['ReversalGate_X2_Y0', 'X0

## pairwise annihilation


In [2]:
N = 3

pa = DSDCircuit(new_toehold_generation=True)
input_signal = pa.new_signal('c', N)
pa.add_global(TH_EXT_NAME, 2)

pa.add_modules(add_reporting=False, modules=[PairwiseAnnihilation(input_signal)])


[[['C0', 'C1', 'Annihilator_0_1'], []],
 [['C0', 'C2', 'Annihilator_0_2'], []],
 [['C1', 'C2', 'Annihilator_1_2'], []]]

In [7]:
FORMAL_GATES_FUELS = False
fcrn = [r.list_format() for r in pa.formal_CRN(include_supporting_species=FORMAL_GATES_FUELS)]
fcrn

[[['C0', 'C1'], []], [['C0', 'C2'], []], [['C1', 'C2'], []]]

In [ ]:
dsd_path = f"../artifacts/verification-test/{N}-PA.pil"
pa.export_PIL(output_file=dsd_path, name_intermediates=True)

enum

In [4]:
dsdio.set_io_objects()
pildata = dsdio.read_pil(os.path.splitext(dsd_path)[0] + "_ENUM-CRNcond_vmax10.pil", is_file=True)

In [5]:
det_icrn, _ = parse_crn(";".join(map(crn_format, pildata["det_reactions"])), is_file = False)
con_icrn, _ = parse_crn(";".join(map(crn_format, pildata["con_reactions"])), is_file = False)

In [8]:
icrn = con_icrn

if not FORMAL_GATES_FUELS:
    # filter out gate and fuel reactions from implementation CRN
    filtered_icrn = []
    for reac, prod in icrn:
        filtered_icrn.append(FormalReaction(reac, prod).excluding(pa._supporting_species).list_format())

    icrn = filtered_icrn

icrn

[[['C1'], ['e142']],
 [['e30'], ['e28', 'C2']],
 [['e211', 'C2'], ['e213']],
 [['e243'], ['e238', 'C2']],
 [['C0', 'e260'], ['e406']],
 [['C0', 'e142'], ['e247']],
 [['e28', 'C2'], ['e30']],
 [['C0', 'e27'], ['e219']],
 [['e219'], ['C0', 'e27']],
 [['e238', 'C2'], ['e243']],
 [['e11'], ['C2', 'e5']],
 [['C0'], ['e238']],
 [['e139', 'C2'], ['e199']],
 [['e353'], ['C0', 'e323']],
 [['e164'], ['C2', 'e145']],
 [['e70', 'C2'], ['e72']],
 [['e318'], ['C0', 'e61']],
 [['e105'], ['C2', 'e60']],
 [['e318'], ['e260', 'C2']],
 [['e418'], ['C0', 'e238']],
 [['C1', 'e142'], ['e181']],
 [['e75'], ['C1', 'e64']],
 [['C0', 'e238'], ['e418']],
 [['e233'], ['e225', 'C2']],
 [['e64'], ['C1']],
 [['e260', 'C1'], ['e287', 'e288']],
 [['e181'], ['C1', 'e142']],
 [['e107'], ['e64', 'C2']],
 [['C1'], ['e64']],
 [['e239', 'C1'], ['e247']],
 [['e145'], ['C2']],
 [['C1', 'e145'], ['e153', 'e152']],
 [['C0', 'e64'], ['e235']],
 [['C2', 'e134'], ['e164']],
 [['e323'], ['C0']],
 [['C0', 'e145'], ['e243']],
 [['C1'

In [9]:

print(f"checking bisimulation equivalence between {len(fcrn)} formal and {len(icrn)} implementation reactions")
v, full_interp = crn_bisimulation_test(icrn=icrn, fcrn=fcrn, interpretation = pa.partial_species_interpretation())
if not v:
    print("NOT equivalent")
else:
    print("EQUIVALENT")
    pprint(full_interp)

checking bisimulation equivalence between 3 formal and 138 implementation reactions
EQUIVALENT
{'C0': ['C0'],
 'C1': ['C1'],
 'C2': ['C2'],
 'e105': ['C2', 'C2'],
 'e107': ['C1', 'C2'],
 'e11': ['C2', 'C2'],
 'e134': ['C2'],
 'e139': ['C1'],
 'e142': ['C1'],
 'e145': ['C2'],
 'e152': [],
 'e153': [],
 'e164': ['C2', 'C2'],
 'e181': ['C1', 'C1'],
 'e199': ['C1', 'C2'],
 'e211': ['C0'],
 'e213': ['C0', 'C2'],
 'e219': ['C0', 'C1'],
 'e225': ['C0'],
 'e233': ['C0', 'C2'],
 'e235': ['C0', 'C1'],
 'e238': ['C0'],
 'e239': ['C0'],
 'e243': ['C0', 'C2'],
 'e247': ['C0', 'C1'],
 'e249': ['C0', 'C2'],
 'e251': ['C0', 'C1'],
 'e260': ['C0'],
 'e27': ['C1'],
 'e28': ['C1'],
 'e287': [],
 'e288': [],
 'e3': ['C2'],
 'e30': ['C1', 'C2'],
 'e310': ['C0', 'C1'],
 'e318': ['C0', 'C2'],
 'e323': ['C0'],
 'e338': [],
 'e339': [],
 'e34': ['C1', 'C2'],
 'e353': ['C0', 'C0'],
 'e406': ['C0', 'C0'],
 'e41': ['C1', 'C1'],
 'e418': ['C0', 'C0'],
 'e5': ['C2'],
 'e60': ['C2'],
 'e61': ['C2'],
 'e64': ['C1'],
